# Day 23 - computational geometry with volumes, or path finding?

* [Day 23](https://adventofcode.com/2018/day/23)

Part 1 is just a small test to see if you can work out the Manhattan distance for 3 dimensions rather than 2. Every additional dimension is just another $+ |dim - dim'|$ addition.

<figure style="float: right; max-width: 10em; margin: 1em">
<img src="https://upload.wikimedia.org/wikipedia/commons/0/07/Octahedron.svg"
     alt="Octahedron illustration from Wikimedia"/>
<figcaption style="font-style: italic; font-size: smaller">

User:Stannered [[GFDL](http://www.gnu.org/copyleft/fdl.html) or [CC-BY-SA-3.0](http://creativecommons.org/licenses/by-sa/3.0/)], [via Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Octahedron.svg)

</figcaption>
</figure>

We are then asked to find the point in the 3D space that is inside the radius of the most nanobots, where the nanobots' radius is not a circle but the Manhattan distance, which is really just an [octahedron](https://en.wikipedia.org/wiki/Octahedron).

We could fill a 3D grid with their volumes, adding 1 to each position within a radius, then finding the position with the highest count that is also closest to $(0, 0, 0)$. However, the size of the coordinate space is *very, very lange*; each of the 3 dimensions covers a range 9 digits small, so the the number of points the coordinates encompass stretches to a number 27 digits long. So this approach is really not feasible.

So our options are to find a different method to solve this (some people have been using a [STM solver](https://en.wikipedia.org/wiki/Satisfiability_modulo_theories) (for Python, that'd be the [`z3-solver` package](https://pypi.org/project/z3-solver/)), but we could also use pathfinding, again.

If you divvy up the large space into cubes, then count how many of the nanobot swarms intersect with that cube, you can prioritise search in that cube over other cubes, subdivide that cube into 8 smaller cubes (dividing the three coordinates in 2) and search further, continuing until you are down to a single point. But you can't be certain you can find the best point by just continuing subdivision like this, just because a cube contains a lot of bot signal coverage intersections doesn't mean there is a single point in that cube with the most overlap.

But we *can* see this as a graph of connected nodes, and apply A* search principles to that graph. The node's distance is the number of subdivisions applied, it's heuristic the number of nanobots whose swarm *don't* intersect the cube (so when all nanobots fit in the cube we have a cost of 0), plus the distance from the origin (favour cubes with lower manhattan distance over those that are further away).



In [1]:
import math
import re
from dataclasses import dataclass
from heapq import heappop, heappush
from itertools import count, product
from operator import attrgetter
from typing import Iterable, Sequence, Tuple

Pos = Tuple[int, int, int]
Range = Tuple[int, int]
Cube = Tuple[Range, Range, Range]

@dataclass(frozen=True)
class Node:
    """Node on the A* search graph"""
    x: Range
    y: Range
    z: Range
    size: int
        
    @property
    def cube(self) -> Cube:
        return self.x, self.y, self.z
    
    @property
    def distance(self) -> Pos:
        # the distance of position nearest to the origin to the origin
        return sum(abs(min(r)) for r in self.cube)
        
    def cost(self, swarm: Sequence['Nanobot']) -> int:
        """Calculate the cost for this node, f(n) = g(n) + h(n)
        
        The cost of this node is the time taken (g) plus
        estimated cost to get to our optimum position (h).
        
        Here we use ignore depth entirely and only use the bots intersecting
        with this cube as the heuristic.
        
        """
        cube = self.cube
        return sum(1 for b in swarm if not b.intersects(cube))

    def priority(self, swarm: Sequence['Nanobot']) -> Tuple[int, int, int]:
        """Priority in the A* search algorithm
        
        First by cost, then by distance from origin, then size
        
        """
        return self.cost(swarm), self.distance, self.size
    
    def transitions(self) -> Iterable['Node']:
        # produce the 8 halved cubes inside this cube
        cls = type(self)
        size = self.size // 2
        if not size:
            return
        for steps in product(range(2), repeat=3):
            cube = ((l + (s * size), min(l + (s + 1) * size - 1, h)) for (l, h), s in zip(self.cube, steps))
            yield cls(*cube, size)

@dataclass(frozen=True, order=True)
class NanoBot:
    x: int
    y: int
    z: int
    r: int
        
    @property
    def pos(self) -> Pos:
        return self.x, self.y, self.z
            
    def distance_from(self, pos: Pos) -> int:
        return sum(abs(sc - oc) for sc, oc in zip(self.pos, pos))
    
    def in_range(self, pos: Pos) -> bool:
        return self.distance_from(pos) <= self.r
    
    def intersects(self, cube: Cube) -> bool:
        dist = 0
        radius = self.r
        for c, (l, h) in zip(self.pos, cube):
            if c < l:
                dist += l - c
            elif c > h:
                dist += c - h 
            if dist > radius:
                return False
        return True
    
_parse_line = re.compile(r'pos=<(-?\d+),(-?\d+),(-?\d+)>,\s*r=(\d+)').search

class TeleportationSwarm:
    def __init__(self, swarm: Sequence['Nanobot']) -> None:
        self.swarm = swarm
        
    @classmethod
    def from_lines(cls, lines: Iterable[str]) -> 'TeleportationSwarm':
        swarm = []
        for line in lines:
            match = _parse_line(line)
            if match is not None:
                swarm.append(NanoBot(*map(int, match.groups())))
        return cls(swarm)
    
    def signal_radius_count(self):
        max_signal = max(self.swarm, key=attrgetter('r'))
        return sum(1 for bot in self.swarm if max_signal.in_range(bot.pos))
    
    def optimal_location_distance(self) -> Pos:
        # Modified A*; we don't need an open or closed set anymore the graph is
        # purely acyclic and directed towards smaller cubes; there is only ever
        # a single path towards any node.
        swarm = self.swarm
        
        minima = tuple(map(min, zip(*map(attrgetter('x', 'y', 'z'), swarm))))
        maxima = tuple(map(max, zip(*map(attrgetter('x', 'y', 'z'), swarm))))
        # starting cube must include 0, 0, 0
        cube = [(min(low, 0), max(high, 0)) for low, high in zip(minima, maxima)]
        # starting size is a power of 2 that can envelop the whole system
        step = 2 ** (int(math.log2(max(maxima))) + 1)
        
        start = Node(*cube, step)
        unique = count()  # tie breaker when costs are equal
        pqueue = [(*start.priority(swarm), next(unique), start)]
        
        while pqueue:
            node = heappop(pqueue)[-1]

            if node.size == 1:
                return node.distance

            for new in node.transitions():
                heappush(pqueue, (*new.priority(swarm), next(unique), new))

In [2]:
assert TeleportationSwarm.from_lines('''\
pos=<0,0,0>, r=4
pos=<1,0,0>, r=1
pos=<4,0,0>, r=3
pos=<0,2,0>, r=1
pos=<0,5,0>, r=3
pos=<0,0,3>, r=1
pos=<1,1,1>, r=1
pos=<1,1,2>, r=1
pos=<1,3,1>, r=1'''.splitlines()).signal_radius_count() == 7
assert TeleportationSwarm.from_lines('''\
pos=<10,12,12>, r=2
pos=<12,14,12>, r=2
pos=<16,12,12>, r=4
pos=<14,14,14>, r=6
pos=<50,50,50>, r=200
pos=<10,10,10>, r=5
'''.splitlines()).optimal_location_distance() == 36

In [3]:
import aocd

data = aocd.get_data(day=23, year=2018)
swarm = TeleportationSwarm.from_lines(data.splitlines())

In [4]:
print('Part 1:', swarm.signal_radius_count())

Part 1: 408


In [5]:
print('Part 2:', swarm.optimal_location_distance())

Part 2: 121167568
